# Automated Trading Strategies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression
plt.style.use("seaborn-v0_8")

## IBKR Data Pull

In [ ]:
from ib_insync import *

In [ ]:
util.startLoop()
ib = IB()
ib.connect()

#### Forex Data Pull

In [ ]:
# Parameters
symbol = "EURUSD"
bar_size = "4 hours"          # e.g. "5 mins", "1 hour", "1 day"
end = "2019-12-31"
what_to_show = "MIDPOINT"   # alternatives: "TRADES", "ASK", etc.

In [ ]:
# 1. Setup contract
contract = Forex(symbol)

In [ ]:
# 2. Compute duration string
end_dt = pd.to_datetime(end)
duration_str = "2 Y"

In [ ]:
# 3. Pull data
data = ib.reqHistoricalData(
    contract,
    endDateTime=end_dt.strftime("%Y%m%d %H:%M:%S"),
    durationStr=duration_str,
    barSizeSetting=bar_size,
    whatToShow=what_to_show,
    useRTH=True,
    formatDate=1
)

In [ ]:
# 4. Convert to df
df = util.df(data)
df.set_index("date", inplace=True)
df = df.dropna()
df = df[["close"]].rename(columns={"close": "price"})
df["returns"] = np.log(df["price"] / df["price"].shift(1))
df = df.dropna()

In [ ]:
ib.disconnect()

### SMA BackTester

__Backtesting & Forward Testing__

In [ ]:
import strategies as strat

In [ ]:
tester = strat.SMABacktester(df, 50, 200, "2005-06-01", "2015-12-31")

In [ ]:
tester.test_strategy()

In [ ]:
tester.results

In [ ]:
tester.plot_results()

In [ ]:
tester.set_parameters(75, 150)

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.optimize_parameters((25, 50, 1), (100, 200, 1))

In [ ]:
tester.plot_results()

In [ ]:
tester.results_overview

In [ ]:
tester.test_strategy()

__Backtesting & Forward Testing__

In [ ]:
#train = strat.SMABacktester("EURUSD=X", 50, 200, "2004-01-01", "2015-12-31")

In [ ]:
#train.optimize_parameters((25, 50, 1), (100, 200, 1)) # optimizing strategy & backtesting ("in-sample")

In [ ]:
#train.plot_results()

In [ ]:
#tester = strat.SMABacktester("EURUSD=X", 46, 137, "2016-01-01", "2020-06-30")

In [ ]:
#tester.test_strategy() # forward testing ("out-sample")

In [ ]:
#tester.plot_results()

__AUD/EUR__

In [ ]:
# tester = strat.SMABacktester("AUDEUR=X", 50, 200, "2004-01-01", "2020-06-30")

In [ ]:
# tester.test_strategy()

In [ ]:
# tester.plot_results()

In [ ]:
# tester.optimize_parameters((25, 50, 1), (100, 200, 1))

In [ ]:
# tester.plot_results()

## ConBacktester

In [ ]:
ptc = 0.00007

In [ ]:
tester = strat.ConBacktester(df, window=3, tc=ptc)

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.optimize_parameter((1, 20, 1))

In [ ]:
tester.results_overview

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results("EURUSD")

## Mean Reversion

In [ ]:
df.info()

In [ ]:
ptc = 0.00007

In [ ]:
# Instantiate the backtester
tester = strat.MeanRevBacktester(df, SMA=30, dev=2, tc=ptc)

In [ ]:
tester = strat.MeanRevBacktester("EURUSD", 30, 2, "2018-01-01", "2019-12-31", ptc)

In [ ]:
tester

In [ ]:
tester.test_strategy()

In [ ]:
tester.results

In [ ]:
tester.plot_results()

In [ ]:
tester.optimize_parameters((25, 100, 1), (1, 5, 1))

In [ ]:
tester.plot_results()

In [ ]:
tester.results

In [ ]:
tester.results_overview

In [ ]:
tester.results_overview.nlargest(10, "performance")

In [ ]:
tester.results_overview.nsmallest(10, "performance")

## Machine Learning 

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
symbol = "EURUSD"

In [ ]:
ptc = 0.00007 * 0

In [ ]:
ml = strat.MLBacktester(df, ptc)

In [ ]:
ml

In [ ]:
ml.data

In [ ]:
ml.test_strategy(train_ratio = 0.7, lags = 5)

In [ ]:
ml.plot_results()

In [ ]:
ml.results

In [ ]:
for lags in range(1, 21):
    print(lags, ml.test_strategy(train_ratio = 0.7, lags = lags))

In [ ]:
ml.results.trades.value_counts()

In [ ]:
hits = np.sign(ml.results.returns * ml.results.pred).value_counts()
hits

In [ ]:
hit_ratio = hits[1.0] / sum(hits)
hit_ratio